In [74]:
#my imports
import pandas as pd
#requests is a popular python libarary for making HTTP requests, such as downloading from URLs
import requests
import urllib.request
import sentencepiece as spm
import tiktoken 
import torch 

## My Variables 
1. vocab size=size of unique characters
2. characters=the sorted list of unique characters 

In [75]:

#got the url by open the raw version of the file in github 
url = "https://raw.githubusercontent.com/yasminho/Shakespeare-gpt/main/input.txt?token=GHSAT0AAAAAACDEOEB7HCMDF43ROSZND5FAZF6U3OA"
try:
    # Download the dataset using urllib
    #downloads the contents from url and saves it as input.txt 
    urllib.request.urlretrieve(url, 'input.txt')
    print("Dataset downloaded successfully.")
#if there is an exception, the code will enter the exception block 
except Exception as e:
    print(f"the dataset did not download: {e}")

#will enter the exception block and print the exception 

Dataset downloaded successfully.


In [76]:
#reading in the dataset 
#'r' means read 
#common encoding for text files 
with open('input.txt', 'r', encoding='utf-8') as f:
    text=f.read()

print(len(text))


1115394


In [77]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [78]:
#list of the set of characters in the text 
characters=sorted(list(set(text)))
#number of unique characters in the text 
size_vocab=len(characters)

print(size_vocab)

65


## Tokenizing

In [79]:
#creating a mappings
#dictionary mapping characters to corresponding integers using enumerate
#enumerate returns an iterator that produces tuples containing index and the element from input sequence 
chars_int_mapping={char:int for int, char in enumerate(characters)}
#dictionary mpping integers to corresponding characters 
int_chars_mapping={int: char for int, char in enumerate(characters)}
#takes in a string and encodes it into a list of integers using our mapping/converts each character in string to corresponding integer value and returns list of integers
encoder=lambda str:[chars_int_mapping[char] for char in str]
#takes in a list of integers and decoes it to a string using our int_chars_mapping
decoder=lambda int_list: ''.join([int_chars_mapping[i] for i in int_list])

#when we use this encoder, we will get a list of numbers between 0-65 because that is the size of vocabulary 
#notice that we get a token for each character
encoder("Hi Yas")



[20, 47, 1, 37, 39, 57]

In [80]:
#tokenizing with tiktoken 
tik_token_encoder=tiktoken.get_encoding('gpt2')
#instead of 65 tokens (our original vocabulary size), it has 50257 tokens 
#when we use this encoder 
tik_token_encoder.n_vocab

#when we use tik_token_encoder, we get a number that anywhere from 0-50257
#notice that we get a token for each word 
tik_token_encoder.encode("Hi")

[17250]

In [81]:
#encoding the text dataset and into a torch.tensor
#torch.tensor is a multi-dimensional array 
#takes in a list or array and converts into a pytorch tensor
#the dtype=torch.long argument tells us the specified data type of the elements in the tensor, which is long (usually for integer data)
encoded_data=torch.tensor(encoder(text), dtype=torch.long)
print(encoded_data.dtype)
print(encoded_data.shape)
print(encoded_data[:1000])

torch.int64
torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

# splitting data
- splitting into train and test set
- 80% will train and 20% will test 

In [82]:
#look into nested-cross-validation
train_length=int(len(encoded_data)*0.80)
train_data=encoded_data[:train_length]
test_data=encoded_data[train_length:]


# Training

In [83]:
#we only work with chunks of the data and train it at chunks at a time 
#the maximum length-called block_size 
#all of these characters follow each other, simulatenosouly make a prediction at all of them 
#8 individual examples: when you are trying to predict 9, we have 8 examples 

context_length=8 
train_data[:context_length+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [84]:

x=train_data[:context_length]
#y includes one after x
#y is the target for each position/the idea that we can predict the second element based on only one input 
y=train_data[1: context_length+1]

#iterating up to context_length because the y: includes the last position

for z in range(context_length):
    #up to and including z/array x is from 0-block_size
    contextual=x[:z+1]
    #not including z/array y is from 1-block_size+1
    prediction=y[z]
    print(f"when input is {contextual} the prediction: {prediction}")




when input is tensor([18]) the prediction: 47
when input is tensor([18, 47]) the prediction: 56
when input is tensor([18, 47, 56]) the prediction: 57
when input is tensor([18, 47, 56, 57]) the prediction: 58
when input is tensor([18, 47, 56, 57, 58]) the prediction: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the prediction: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the prediction: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the prediction: 58


In [86]:
#working with multiple chunks/batchs together
#working with batch dimension 

#random number generator/changes this 
torch.manual_seed(1337) 
#number of different independent sequences that we will process in parallel 
head_size=4 
#maximum contextual length for the predictions 
contextual_length=8

#function to do multi-headed attention 


def batch(input):
    if input=='train':
        data=train_data
    else: 
        data=test_data
    index=torch.randint(len(data)-contextual_length, (head_size,)) #grabbing head_size number of random offsets/index is going to be #defined by head_size randomly generated between the length of data and our contextual_length
    contextual=torch.stack([data[i:i+contextual_length] for i in index]) #all become a row in a 4 by 8 tensor 
    prediction=torch.stack([data[i+1:i+contextual_length+1] for i in index])
    return contextual, prediction

cont, pred=batch('train')
print('inputs:')
print(cont.shape)
print(cont)
print('prediction:')
print(pred.shape)
print(pred)

print('----')

for b in range(head_size): #number of heads
    for c in range(contextual_length): 
        context=cont[b, :c+1]
        prediction=pred[b, c]
        print(f"when input is {context.tolist()} the prediction: {prediction}") 


#we are getting 4 rows (represent the heads) 
#we get 8 columns (represent the contextual part of it)


    
    

inputs:
torch.Size([4, 8])
tensor([[58, 63,  8,  0,  0, 19, 24, 27],
        [39, 59, 45, 46, 58,  1, 46, 43],
        [49, 43, 57,  1, 53, 50, 42,  1],
        [52, 41, 47, 43, 52, 58,  1, 56]])
prediction:
torch.Size([4, 8])
tensor([[63,  8,  0,  0, 19, 24, 27, 33],
        [59, 45, 46, 58,  1, 46, 43,  1],
        [43, 57,  1, 53, 50, 42,  1, 46],
        [41, 47, 43, 52, 58,  1, 56, 47]])
----
when input is [58] the prediction: 63
when input is [58, 63] the prediction: 8
when input is [58, 63, 8] the prediction: 0
when input is [58, 63, 8, 0] the prediction: 0
when input is [58, 63, 8, 0, 0] the prediction: 19
when input is [58, 63, 8, 0, 0, 19] the prediction: 24
when input is [58, 63, 8, 0, 0, 19, 24] the prediction: 27
when input is [58, 63, 8, 0, 0, 19, 24, 27] the prediction: 33
when input is [39] the prediction: 59
when input is [39, 59] the prediction: 45
when input is [39, 59, 45] the prediction: 46
when input is [39, 59, 45, 46] the prediction: 58
when input is [39, 59, 45